In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-kernel-common-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-304-updates-dev' for glob 'nvidia*'
Note, selecting 'nvidia-compute-utils-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-libopencl1-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-340-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-source-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-331-updates-uvm' for glob 'nvidi

In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

--2020-12-04 14:28:07--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.0.24
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.0.24|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?zf-mJaaunIETz2jx7aohQSPiGy8pyuXrf7ha2pSDc0CGyVKxkNXyMg37tHaQcfNWQgyw9Byo6GHi_FG-3Zt2Ly9T5ZBBjlASGDQX5wqwEHklL07j_FJJAZAikodCimckyuzzG43ciR1QubnoQojBAz_pfb9JpgkZMZfsxT4pbKDPOvJFJ_UljlKNjllCuY2qy3uk4huHBA-UlaRoJV0 [following]
--2020-12-04 14:28:08--  https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?zf-mJaaunIETz2jx7aohQSPiGy8pyuXrf7ha2pSDc0CGyVKxkNXyMg37tHaQcfNWQgyw9Byo6GHi_FG-3Zt2Ly9T5ZBBjlASGDQX5wqwEHklL07j_FJJAZAikodC

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Wed_Apr_11_23:16:29_CDT_2018
Cuda compilation tools, release 9.2, V9.2.88


In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-5dw_qrmi
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-5dw_qrmi
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=bc015eb629b2347b88a33d3b3576985334b2645de679a452f4f70d3d32acaf00
  Stored in directory: /tmp/pip-ephem-wheel-cache-r7rtiuh6/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

directory /content/src already exists
Out bin /content/result.out


In [ ]:
%%cu 
#include <iostream> 
    int
    main() 
{ 
    std::cout << "Welcome To GeeksforGeeks\n"; 
    return 0; 
} 


Welcome To GeeksforGeeks



In [ ]:
%%cu

#include <iostream>
#include <cstdio>
#include <string>
#define TOGG(k) ( ( ( (k) & 1 ) << 5 ) )
#define RAN(charac) ( 65 + ( charac % 26 ) + TOGG ( charac ) ) 
using namespace std;

__global__ 
void RunLengthEncodingComputation (char *orig, int *_encoXst, int n) {
    int index = ( (blockIdx.x * blockDim.x) + threadIdx.x );
    index <<= 7;

    if(orig[index] == orig[index-1]) 
        while(index < n && orig[index] == orig[index-1])
            ++index;
    
    for (int i = index; i < fminf(index + 128, n); )
    {
        char temp = orig[i];
        int t_ = i;
        while (i < n && temp == orig[i])
            ++i;
        _encoXst[t_] = i;
    }
}

int main()  {
    int n;
    cin >> n;
    char *s = new char[n];
    int i = 0;

    for( ; i < n; ) {
        char in = RAN ( rand() );
        int loop = rand() & 63, k = i;
        while(i < min(k+loop, n))
            s[i++] = in;
    }
    s[i] = '\0';
    fprintf(fopen("input.txt", "w"), "%s", s);
    char *cudas;
    int *_encoXst, *_inter = new int[n];

    int threads = (1 << 7);
    int blocks = ( ( n>>14 ) + ( ( n & ( (1<<14)-1 ) ) != 0 )  );

    cout << threads << " : " << blocks << endl;

    cudaMalloc (&cudas, n*sizeof(char));
    cudaMalloc (&cudas, n*sizeof(char));
    cudaMalloc (&_encoXst, n*sizeof(int));

    cudaMemcpy (cudas, s, n*sizeof(char), cudaMemcpyHostToDevice);

    RunLengthEncodingComputation <<<blocks, threads>>> (cudas, _encoXst, n);
    cudaDeviceSynchronize();
    cudaMemcpy(_inter, _encoXst, n*sizeof(int), cudaMemcpyDeviceToHost);

    string ans;
    int sum = 0;
    
    for(int i = 0; i < n; i = _inter[i]) {
        ans += s[i] + to_string(_inter[i]-i);
        sum += _inter[i]-i;
    }
    //cout << ans << endl;
    fprintf(fopen("output.txt", "w"), "%s", ans.c_str());
    int length_ans = ans.length();
    printf("Length: %d\nCompressed Length: %d\nCompression Achievement: %f\n", sum, length_ans, (float)(sum+length_ans-1)/(length_ans));
}

128 : 2
Length: 32535
Compressed Length: 2843
Compression Achievement: 12.443545

